In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.impute import SimpleImputer

from datetime import timedelta




In [2]:
feng_trainning_set = '/usr/src/tp2-1/data-in/210301_feng_Train_TP2_Datos_2020-2C.csv'
feng_eval_set = '/usr/src/tp2-1/data-in/210301_feng_Test_TP2_Datos_2020-2C.csv'

train_source_file = feng_trainning_set
test_source_file = feng_eval_set

kaggle_predict_file =  '/usr/src/tp2-1/data-out/2100301_random_forest_TimeSeriesSplit.csv'




trainning_set = '/usr/src/tp2-1/data-in/Train_TP2_Datos_2020-2C.csv'
eval_set = '/usr/src/tp2-1/data-in/Test_TP2_Datos_2020-2C.csv'

feng_trainning_set = '/usr/src/tp2-1/data-in/210301_feng_Train_TP2_Datos_2020-2C.csv'
feng_eval_set = '/usr/src/tp2-1/data-in/210301_feng_Test_TP2_Datos_2020-2C.csv'

In [3]:
def normaliza_df(df, onehotencoder = None):
    
    # drops -------------------------------------------------
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # N/R
    df = df.drop(['ID'], axis=1)
    
    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)
    
    # esta columna es equivalente a oportunity_id
    df = df.drop(['Opportunity_Name'], axis=1)
    
    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    my_imputer = SimpleImputer(strategy='most_frequent')
    df_rx = pd.DataFrame(my_imputer.fit_transform(df))
    df_rx.columns = df.columns
    df = df_rx 
    
    # Total_Taxable_Amount = Sum rows[Total_Amount]

    numeric_cols = ['Opportunity_ID', 'TRF', 'ASP', 'ASP_(converted)', 'Total_Amount', 'Total_Taxable_Amount']
    if 'Stage' in df.columns:
        numeric_cols.append('Stage')
    else:
        print('no la agrego!')
        
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)
    
    df['Last_Modified_Date'] = pd.to_datetime(df['Last_Modified_Date'], format="%m/%d/%Y")
    
    def get_factor_estim_periodo(currency, date_in_row):
        
        
        date_window_begin = date_in_row - timedelta(days=30)
        date_window_end = date_in_row + timedelta(days=30)
        #print('buscando para: ', currency, date_in_row, date_window_begin, date_window_end)
        
        mask = ((df['Last_Modified_Date'] >= date_window_begin) | (df['Last_Modified_Date'] >= date_window_end)) & (df['ASP_Currency'] == currency) & (df['ASP_max'] > 0)
        #mask = (df['Last_Modified_Date'] > date_window_begin) & (df['Last_Modified_Date'] <= date_window_end)
        sdf = df.loc[mask]
        factor = sdf['ASP_(converted)_max'].mean() / sdf['ASP_max'].mean() 
        #print(factor, sdf['ASP_(converted)_max'].mean(), sdf['ASP_max'].mean() )
        #print(sdf[['ASP_Currency', 'ASP_(converted)_max', 'ASP_max']].head(10))
        #sub_df = df.loc[interval:'ASP_max']
        #print(sub_df)
        #print(sub_df.mean)
        
        
        # busca el valor pronedio de asp y asp_convert para $currency
        # dentro de una cierta ventana de tiempo entorno a $date_in_row
        
        return factor
        
        #mask = (df['date'] > start_date) & (df['date'] <= end_date)
        #df.loc[mask]
        
    def define_factor(a, b, currency, date_in_row):
        if (a == 0) or (a == np.nan) or  (b == 0) or (b == np.nan):
            return get_factor_estim_periodo(currency, date_in_row)
        else:
            return a / b
        
    '''
    if ((row['ASP_max'] == 0) | (row['ASP_max'] == np.nan)):
            return -1
        else:
            return row["ASP_(converted)_max"] / row["ASP_max"]
    ''' 
        
    # preparamos la columna Total_Taxable_Amount para ser aplanada
    df["Total_Amount_sum"] = df.groupby("Opportunity_ID")["Total_Amount"].transform("sum")
    
    df["ASP_max"] = df.groupby("Opportunity_ID")["ASP"].transform("max")
    df["ASP_(converted)_max"] = df.groupby("Opportunity_ID")["ASP_(converted)"].transform("max")
    
    df["rel_cnv_currency"] = df.apply(lambda x: define_factor(x["ASP_(converted)_max"], x['ASP_max'], x['ASP_Currency'], x['Last_Modified_Date']) , axis=1)
    

    df = df.drop(['ASP'], axis=1)
    df = df.drop(['ASP_(converted)'], axis=1)
    
    #return df, OneHotEncoder(handle_unknown = 'ignore')
    
    
    #df["rel_cnv_currency"] = df["ASP_(converted)_max"] / df["ASP_max"]
    
    # @todo hay un solo registro que queda anomalo, eliminarlo
    #  >>> df.loc[df_train['ASP_(converted)'] / df['ASP'] > 2, :] = 0
    
    
    #df = df.drop(['Total_Taxable_Amount'], axis=1)
    #df = df.drop(['Total_Amount_Currency'], axis=1)
    df["Total_Amount_sum_usd"] = df["rel_cnv_currency"] * df["Total_Amount_sum"]
    df = df.drop(['Total_Amount_sum'], axis=1)
    df["Total_Amount_usd"] = df["rel_cnv_currency"] * df["Total_Amount"]
    df = df.drop(['Total_Amount'], axis=1)
    df["Total_Taxable_Amount_usd"] = df["rel_cnv_currency"] * df["Total_Taxable_Amount"]
    df = df.drop(['Total_Taxable_Amount'], axis=1)
    
    df = df.drop(['Total_Amount_Currency'], axis=1)
    df = df.drop(['Total_Taxable_Amount_Currency'], axis=1)
    df = df.drop(['ASP_Currency'], axis=1)
    df = df.drop(['ASP_(converted)_Currency'], axis=1)
    
    
    #return df, OneHotEncoder(handle_unknown = 'ignore')
   
    
    # todo: convertir estas en una ventana de tiempo
    df['Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], format="%m/%d/%Y")
    df['Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], format="%m/%d/%Y")
    
    df['Planned_Delivery_Date_diff'] = (df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']).dt.days
    
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)
    
    # @todo: no estoy seguro si esto aporta
    df = df.drop(['Quote_Expiry_Date'], axis=1)
    
    # @todo: convertirla a una diferencia de dias?
    df = df.drop(['Last_Modified_Date'], axis=1)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")
    
    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    
    
    #------------------------------------------
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')
    #------------------------------------------
    
    # df = df.drop(4907, axis = 0)
    # ndf.loc[ndf.rel_cnv_currency > 2,:]

    
    numeric_cols_rv = ['ASP_max', 'ASP_(converted)_max', 'rel_cnv_currency', 
                       'Total_Amount_sum_usd', 'Total_Amount_usd', 'Total_Taxable_Amount_usd', 'Planned_Delivery_Date_diff', 'Opportunity_ID']
    if 'Stage' in df.columns:
        numeric_cols.append('Stage')
    
    df[numeric_cols_rv] = df[numeric_cols_rv].apply(pd.to_numeric)
    
    ###
    
    #df[numeric_cols_rv] = df[numeric_cols_rv].apply(pd.to_numeric)
    #df.dropna()
    
    #return df, OneHotEncoder(handle_unknown = 'ignore')

    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 

    
    if 'Stage' in df.columns:
        onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
        onehotencoder.fit(df[categoric_cols])
    
    cat_rel = 0
    all_col_names = []
    for cat in onehotencoder.categories_:
        for col in cat:
            all_col_names.append(categoric_cols[cat_rel] + '_' + str(col))
        cat_rel = cat_rel + 1
    
    categorical = pd.DataFrame(onehotencoder.transform(df[categoric_cols]).toarray(), columns=all_col_names)
    
    categorical = categorical.reset_index()
    df = df.reset_index()
    
    frames_to_concat = [categorical, df[numeric_cols], df[numeric_cols_2]]
    df_r =  pd.concat(frames_to_concat, axis=1)
       
    df_r = df_r.drop(columns = 'index')
    
    #@todo sigue dando problemas este campo armado: Planned_Delivery_Date_diff
    df_r = df_r.fillna(0)
    
    return df_r, onehotencoder


In [18]:
df_train = pd.read_csv(trainning_set )

# como vamos a hacer un merge, no queremos perder filas solo por que
# alguna posicion tiene algun dato faltante
# df_train = df_train.fillna(0) >>>  @todo revisar que pasa con esto, lo resuleve el xgboost o lo normalizo antes?

In [19]:
#df_train.describe()

In [20]:
#df_train.loc[df_train['ASP_(converted)'] / df_train['ASP'] > 1.5, ['ASP', 'ASP_(converted)']]

In [21]:
#df_train.info()
cols_with_missing = [col for col in df_train.columns
                     if df_train[col].isnull().any()]

cols_with_missing

['ASP', 'ASP_(converted)', 'Total_Amount']

In [22]:
#df_train[['ASP', 'ASP']]

In [23]:
#df_train.loc[df_train['Opportunity_ID'] == 10]

In [24]:
[ndf, onehotencoder] = normaliza_df(df_train)

#ndf = ndf.drop(4907, axis = 0)
#ndf.loc[ndf.rel_cnv_currency > 2,:]

In [25]:
#fillna(0)

In [26]:
#df_train.info()
cols_with_missing = [col for col in ndf.columns
                     if ndf[col].isnull().any()]

cols_with_missing

[]

In [27]:
#ndf.dtypes
#ndf.describe()
#ndf.loc[ndf.ASP_max == 0, :]
#ndf.Total_Amount_Currency.value_counts()

#asp_val = (ndf.loc[ndf.Total_Amount_Currency == 'EUR','ASP_max']).mean()
#asp_cnv_val = (ndf.loc[ndf.Total_Amount_Currency == 'EUR','ASP_(converted)_max']).mean()
#ndf.loc[ndf['Opportunity_ID'] == 10]


In [28]:
ndf2 = ndf.groupby('Opportunity_ID').max()
ndf2 = ndf2.reset_index()
#ndf2 = df.drop(columns = 'Opportunity_ID')
#ndf2 = ndf2.fillna(0)

In [29]:
print(ndf2.shape)
#ndf2 = ndf2.dropna()
#print(ndf2.shape)

(9791, 2865)


In [30]:
ndf2.to_csv(feng_trainning_set)

# proceso el set de test

In [31]:
df_test = pd.read_csv(eval_set)
[ndft, _] = normaliza_df(df_test, onehotencoder)

ndft2 = ndft.groupby('Opportunity_ID').max()
ndft2 = ndft2.reset_index()

print(ndft2.shape)

ndft2.to_csv(feng_eval_set)

no la agrego!


# ahora si juego con el set para ver un poco mas la relacion entre features

In [16]:
# guarda con esto que altera el Set (por eso lo grabe antes)
X = ndf2
y = X.pop('Stage')
ID = X.pop('Opportunity_ID')

In [249]:
X.describe()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Territory_Albania,Territory_Armenia,Territory_Australia,Territory_Austria,Territory_Belarus,...,Prod_Category_A_Prod_Category_A_None,ASP_max,ASP_(converted)_max,rel_cnv_currency,Total_Amount_sum_usd,Total_Amount_usd,Total_Taxable_Amount_usd,Planned_Delivery_Date_diff,Quote_Type,TRF
count,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,9791.000000,...,9791.0,9791.000000,9791.000000,9791.000000,9.791000e+03,9.791000e+03,9.791000e+03,9791.000000,9791.000000,9791.000000
mean,0.212338,0.250434,0.330610,0.192524,0.014095,0.000204,0.000102,0.058932,0.007354,0.000306,...,1.0,9.365970,0.451011,0.872219,1.719144e+06,1.359723e+06,1.621077e+06,29.946584,0.013277,3.133184
std,0.408984,0.433285,0.470457,0.394302,0.117887,0.014292,0.010106,0.235509,0.085442,0.017503,...,0.0,21.551504,0.894175,0.388130,8.459134e+06,7.339129e+06,1.599115e+07,703.568543,0.114466,14.838808
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.008866,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.370000,0.380000,1.000000,4.730625e+04,4.189500e+04,2.850000e+04,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.410000,0.420000,1.000000,1.277966e+05,1.075900e+05,1.031093e+05,6.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.530000,0.494470,1.131086,5.139525e+05,4.527174e+05,4.034625e+05,28.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.0,80.000000,67.000000,1.318056,3.150001e+08,3.150001e+08,1.356339e+09,69474.000000,1.000000,500.000000


In [250]:
from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y)
#

In [257]:
mi_scores.nlargest(50)

Total_Amount_usd                         0.176835
Total_Amount_sum_usd                     0.152148
Total_Taxable_Amount_usd                 0.137605
TRF                                      0.126928
ASP_(converted)_max                      0.087657
Planned_Delivery_Date_diff               0.079384
ASP_max                                  0.071614
Product_Type_None                        0.052618
Currency_None                            0.049277
rel_cnv_currency                         0.049158
Brand_None                               0.048605
Opportunity_Type_Opportunity_Type_19     0.044689
Account_Type_Account_Type_0              0.038647
Bureaucratic_Code_Bureaucratic_Code_4    0.037544
Product_Category_B_None                  0.037537
Price_None                               0.036565
Size_None                                0.036172
Size_Other                               0.035658
Bureaucratic_Code_Bureaucratic_Code_5    0.033828
Price_Other                              0.032830


In [258]:
mi_scores.nlargest(50).index

Index(['Total_Amount_usd', 'Total_Amount_sum_usd', 'Total_Taxable_Amount_usd',
       'TRF', 'ASP_(converted)_max', 'Planned_Delivery_Date_diff', 'ASP_max',
       'Product_Type_None', 'Currency_None', 'rel_cnv_currency', 'Brand_None',
       'Opportunity_Type_Opportunity_Type_19', 'Account_Type_Account_Type_0',
       'Bureaucratic_Code_Bureaucratic_Code_4', 'Product_Category_B_None',
       'Price_None', 'Size_None', 'Size_Other',
       'Bureaucratic_Code_Bureaucratic_Code_5', 'Price_Other',
       'Opportunity_Type_Opportunity_Type_7', 'Brand_Other', 'Size_Size_3',
       'Account_Name_Account_Name_1662', 'Account_Name_Account_Name_594',
       'Account_Name_Account_Name_1293', 'Account_Name_Account_Name_2063',
       'Account_Type_Account_Type_5', 'Account_Name_Account_Name_1972',
       'Product_Category_B_Other', 'Bureaucratic_Code_0_Approval_1',
       'Account_Name_Account_Name_393', 'Bureaucratic_Code_0_Approval_0',
       'Territory_Solomon Islands', 'Account_Name_Account_Na

In [259]:
top_feat = ['Total_Amount_usd', 'Total_Amount_sum_usd', 'Total_Taxable_Amount_usd',
       'TRF', 'Planned_Delivery_Date_diff', 'Product_Type', 'rel_cnv_currency', 'Brand',
       'Opportunity_Type_Opportunity_Type', 'Account_Type',
       'Bureaucratic_Code', 'Product_Category_B_None',
       'Price_None', 'Size_None', 'Size_Other', 'Price_Other',
       'Size_Size_3',
       'Account_Name',
       'Product_Category_B_Other', 'Bureaucratic_Code_0_Approval',
       'Account_Name',
       'Territory', 'Account_Name_Account_Name_84',
       'Currency_USD', 'Product_Name',
       'Account_Name_Account_Name_1470', 'Account_Name_Account_Name_111',
       'Product_Family_Product_Family_19',
       'Account_Name', 'Price_0.28',
       'Opportunity_Owner_Person_Name', 
       'Last_Modified_By_Person_Name']

In [284]:
ndf2.shape

(9791, 2863)

In [285]:
cols_with_missing = [col for col in ndf2.columns
                     if ndf2[col].isnull().any()]

cols_with_missing

[]